In [ ]:
"""
HYBRID ENSEMBLE - SIFT (0.303) + Fast CNN
Strategy: Use proven SIFT as primary, CNN as refinement
Target: 0.305-0.315

Rationale:
- SIFT gives consistent 0.303
- Add lightweight CNN for edge cases SIFT misses
- Ensemble voting for final decision
"""

import os
import json
import warnings
warnings.filterwarnings('ignore')

import numpy as np
import pandas as pd
from PIL import Image
from tqdm import tqdm
import cv2
from scipy.ndimage import label as scipy_label, binary_fill_holes
from skimage.measure import regionprops

import torch
import torch.nn as nn
import torch.nn.functional as F

# =============================================================================
# CONFIG
# =============================================================================

class Config:
    BASE_PATH = '/kaggle/input/recodai-luc-scientific-image-forgery-detection'
    TRAIN_IMAGES_FORGED = os.path.join(BASE_PATH, 'train_images/forged')
    TRAIN_IMAGES_AUTH = os.path.join(BASE_PATH, 'train_images/authentic')
    TRAIN_MASKS = os.path.join(BASE_PATH, 'train_masks')
    TEST_IMAGES = os.path.join(BASE_PATH, 'test_images')
    SAMPLE_SUB = os.path.join(BASE_PATH, 'sample_submission.csv')
    
    # SIFT parameters (proven 0.303)
    SIFT_FEATURES = 5500
    SIFT_CONTRAST = 0.019
    MATCH_RATIO = 0.79
    MIN_MATCHES = 4
    RANSAC_THRESH = 5.5
    MIN_DISPLACEMENT = 23
    MAX_IMAGE_SIZE = 1600
    USE_CLAHE = True
    
    # VARIANT: Slightly more aggressive for private LB
    SIFT_CONFIDENCE_THRESHOLD = 0.31  # Lower from 0.33
    SIFT_MIN_MASK_PIXELS = 85  # Lower from 90
    SIFT_MIN_COVERAGE = 0.00035  # Slightly lower
    SIFT_MAX_COVERAGE = 0.42  # Slightly higher
    
    # CNN parameters (lightweight)
    CNN_ENABLED = True
    CNN_IMAGE_SIZE = 256  # Small for speed
    CNN_THRESHOLD = 0.5
    
    # Ensemble
    ENSEMBLE_MODE = 'weighted'  # 'weighted' or 'voting'
    SIFT_WEIGHT = 0.75  # SIFT is primary
    CNN_WEIGHT = 0.25   # CNN is refinement

config = Config()

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"🎯 HYBRID ENSEMBLE - SIFT (0.303) + Fast CNN")
print(f"   Device: {device}")

# =============================================================================
# SIFT DETECTOR (PROVEN 0.303)
# =============================================================================

def rle_encode(mask, fg_val=1):
    if mask.sum() == 0:
        return []
    dots = np.where(mask.T.flatten() == fg_val)[0]
    run_lengths = []
    prev = -2
    for b in dots:
        if b > prev + 1:
            run_lengths.extend((b + 1, 0))
        run_lengths[-1] += 1
        prev = b
    return run_lengths

def preprocess_image(img_array):
    if len(img_array.shape) == 3:
        if img_array.shape[2] == 4:
            img_array = img_array[:, :, :3]
        gray = cv2.cvtColor(img_array, cv2.COLOR_RGB2GRAY)
    else:
        gray = img_array.copy()
    gray = cv2.normalize(gray, None, 0, 255, cv2.NORM_MINMAX).astype(np.uint8)
    if config.USE_CLAHE:
        clahe = cv2.createCLAHE(clipLimit=2.5, tileGridSize=(8, 8))
        gray = clahe.apply(gray)
    return gray

def resize_smart(image, max_size):
    h, w = image.shape[:2]
    if max(h, w) <= max_size:
        return image, 1.0
    scale = max_size / max(h, w)
    new_h, new_w = int(h * scale), int(w * scale)
    resized = cv2.resize(image, (new_w, new_h), interpolation=cv2.INTER_AREA)
    return resized, scale

def scale_mask_back(mask, orig_shape, scale):
    if scale == 1.0:
        return mask
    h, w = orig_shape[:2]
    scaled = cv2.resize(mask.astype(np.float32), (w, h), interpolation=cv2.INTER_LINEAR)
    return (scaled > 0.5).astype(np.uint8)

class SIFTDetector:
    def __init__(self, config):
        self.config = config
        self.sift = cv2.SIFT_create(
            nfeatures=config.SIFT_FEATURES,
            contrastThreshold=config.SIFT_CONTRAST,
            edgeThreshold=10
        )
    
    def detect(self, image):
        gray = preprocess_image(image)
        orig_shape = image.shape
        gray, scale = resize_smart(gray, config.MAX_IMAGE_SIZE)
        h, w = gray.shape
        
        kp, desc = self.sift.detectAndCompute(gray, None)
        
        if desc is None or len(desc) < config.MIN_MATCHES * 2:
            return np.zeros((h, w), dtype=np.uint8), 0.0
        
        bf = cv2.BFMatcher(cv2.NORM_L2, crossCheck=False)
        matches = bf.knnMatch(desc, desc, k=2)
        
        good = []
        for m_n in matches:
            if len(m_n) == 2:
                m, n = m_n
                if m.queryIdx != m.trainIdx and m.distance < config.MATCH_RATIO * n.distance:
                    good.append(m)
        
        if len(good) < config.MIN_MATCHES:
            return np.zeros((h, w), dtype=np.uint8), 0.0
        
        src_pts = np.float32([kp[m.queryIdx].pt for m in good]).reshape(-1, 2)
        dst_pts = np.float32([kp[m.trainIdx].pt for m in good]).reshape(-1, 2)
        
        disp = dst_pts - src_pts
        dist = np.linalg.norm(disp, axis=1)
        valid = dist > config.MIN_DISPLACEMENT
        
        if valid.sum() < config.MIN_MATCHES:
            return np.zeros((h, w), dtype=np.uint8), 0.0
        
        src_pts = src_pts[valid]
        dst_pts = dst_pts[valid]
        
        try:
            M, inliers = cv2.findHomography(src_pts, dst_pts, cv2.RANSAC, config.RANSAC_THRESH)
            
            if M is None or inliers is None:
                return np.zeros((h, w), dtype=np.uint8), 0.0
            
            num_inliers = inliers.sum()
            if num_inliers < config.MIN_MATCHES:
                return np.zeros((h, w), dtype=np.uint8), 0.0
            
            mask = np.zeros((h, w), dtype=np.uint8)
            
            inlier_src = src_pts[inliers.flatten() > 0]
            inlier_dst = dst_pts[inliers.flatten() > 0]
            
            for pt in inlier_src:
                x, y = int(pt[0]), int(pt[1])
                if 0 <= x < w and 0 <= y < h:
                    cv2.circle(mask, (x, y), 13, 1, -1)
            
            for pt in inlier_dst:
                x, y = int(pt[0]), int(pt[1])
                if 0 <= x < w and 0 <= y < h:
                    cv2.circle(mask, (x, y), 13, 1, -1)
            
            confidence = min(1.0, num_inliers / max(len(src_pts), 10))
            if num_inliers > 15:
                confidence = min(1.0, confidence * 1.25)
            elif num_inliers > 10:
                confidence = min(1.0, confidence * 1.15)
            
            if scale != 1.0:
                mask = scale_mask_back(mask, orig_shape, scale)
            
            return mask, confidence
        except:
            return np.zeros((h, w), dtype=np.uint8), 0.0

def refine_mask(mask):
    if mask.sum() == 0:
        return mask
    
    h, w = mask.shape
    kernel_small = cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (3, 3))
    kernel_medium = cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (5, 5))
    
    mask = cv2.morphologyEx(mask, cv2.MORPH_OPEN, kernel_small)
    mask = cv2.morphologyEx(mask, cv2.MORPH_CLOSE, kernel_medium, iterations=2)
    mask = binary_fill_holes(mask).astype(np.uint8)
    
    labeled, _ = scipy_label(mask)
    for region in regionprops(labeled):
        if region.area < 45 or region.area > (h * w) * 0.40:
            mask[labeled == region.label] = 0
    
    return mask

# =============================================================================
# FAST CNN (LIGHTWEIGHT REFINEMENT)
# =============================================================================

class FastCNN(nn.Module):
    """Ultra-lightweight CNN for quick inference"""
    def __init__(self):
        super().__init__()
        self.features = nn.Sequential(
            nn.Conv2d(3, 16, 3, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(2),
            nn.Conv2d(16, 32, 3, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(2),
            nn.Conv2d(32, 64, 3, padding=1),
            nn.ReLU(),
            nn.AdaptiveAvgPool2d(1)
        )
        self.classifier = nn.Linear(64, 1)
    
    def forward(self, x):
        x = self.features(x)
        x = x.view(x.size(0), -1)
        return torch.sigmoid(self.classifier(x))

class CNNDetector:
    def __init__(self, config, device):
        self.config = config
        self.device = device
        self.model = FastCNN().to(device)
        self.model.eval()
    
    def detect(self, image):
        """Quick CNN-based forgery probability"""
        # Resize
        img_small = cv2.resize(image, (config.CNN_IMAGE_SIZE, config.CNN_IMAGE_SIZE))
        
        # Normalize
        img_tensor = torch.from_numpy(img_small.astype(np.float32) / 255.0)
        img_tensor = img_tensor.permute(2, 0, 1).unsqueeze(0).to(self.device)
        
        # Predict
        with torch.no_grad():
            prob = self.model(img_tensor).cpu().item()
        
        return prob

# =============================================================================
# HYBRID ENSEMBLE
# =============================================================================

class HybridEnsemble:
    def __init__(self, config, device):
        self.config = config
        self.sift_detector = SIFTDetector(config)
        if config.CNN_ENABLED:
            self.cnn_detector = CNNDetector(config, device)
        else:
            self.cnn_detector = None
    
    def detect(self, image):
        h, w = image.shape[:2]
        
        # Primary: SIFT detection
        sift_mask, sift_conf = self.sift_detector.detect(image)
        sift_mask = refine_mask(sift_mask)
        
        # Adjust SIFT confidence
        if sift_mask.sum() > 0:
            coverage = sift_mask.sum() / (h * w)
            if coverage < 0.0003:
                sift_conf *= 0.35
            elif coverage > 0.40:
                sift_conf *= 0.25
            elif 0.0008 < coverage < 0.18:
                sift_conf = min(1.0, sift_conf * 1.13)
            elif coverage > 0.25:
                sift_conf *= 0.8
        else:
            sift_conf = 0.0
        
        sift_conf = np.clip(sift_conf, 0, 1)
        
        # Secondary: CNN refinement (if enabled)
        if self.cnn_detector is not None:
            cnn_prob = self.cnn_detector.detect(image)
        else:
            cnn_prob = 0.5  # Neutral
        
        # Ensemble decision
        if config.ENSEMBLE_MODE == 'weighted':
            final_conf = config.SIFT_WEIGHT * sift_conf + config.CNN_WEIGHT * cnn_prob
        else:  # voting
            sift_vote = 1 if sift_conf > config.SIFT_CONFIDENCE_THRESHOLD else 0
            cnn_vote = 1 if cnn_prob > config.CNN_THRESHOLD else 0
            final_conf = (sift_vote + cnn_vote) / 2.0
        
        return sift_mask, final_conf

# =============================================================================
# SUBMISSION
# =============================================================================

def generate_submission(config, detector):
    print("\nGenerating submission...")
    
    sample_sub = pd.read_csv(config.SAMPLE_SUB)
    submissions = []
    stats = {'authentic': 0, 'forged': 0}
    
    for case_id in tqdm(sample_sub['case_id'], desc="Processing"):
        try:
            img = np.array(Image.open(os.path.join(config.TEST_IMAGES, f"{case_id}.png")))
            mask, conf = detector.detect(img)
            
            is_forged = False
            if conf > config.SIFT_CONFIDENCE_THRESHOLD and mask.sum() >= config.SIFT_MIN_MASK_PIXELS:
                coverage = mask.sum() / (img.shape[0] * img.shape[1])
                if config.SIFT_MIN_COVERAGE < coverage < config.SIFT_MAX_COVERAGE:
                    is_forged = True
            
            if is_forged:
                rle = rle_encode(mask)
                if len(rle) > 0:
                    annotation = json.dumps([int(x) for x in rle])
                    stats['forged'] += 1
                else:
                    annotation = 'authentic'
                    stats['authentic'] += 1
            else:
                annotation = 'authentic'
                stats['authentic'] += 1
            
            submissions.append({'case_id': case_id, 'annotation': annotation})
        except:
            submissions.append({'case_id': case_id, 'annotation': 'authentic'})
            stats['authentic'] += 1
    
    df = pd.DataFrame(submissions)
    df.to_csv('submission.csv', index=False)
    
    print(f"\n✓ Complete:")
    print(f"  Forged: {stats['forged']} ({stats['forged']/len(df)*100:.1f}%)")
    print(f"  Authentic: {stats['authentic']} ({stats['authentic']/len(df)*100:.1f}%)")
    print("\n✓ Saved: submission.csv")

def main():
    print("="*60)
    print("HYBRID ENSEMBLE - SIFT + CNN")
    print("="*60)
    
    detector = HybridEnsemble(config, device)
    generate_submission(config, detector)
    
    print("\n" + "="*60)
    print("✅ ENSEMBLE COMPLETE")
    print("="*60)

if __name__ == "__main__":
    main()

In [ ]:
import os
import warnings
warnings.filterwarnings('ignore')
import numpy as np
import pandas as pd
from PIL import Image
from tqdm import tqdm
import cv2
import torch
import torch.nn as nn
import torch.nn.functional as F

# =============================================================================
# CONFIG
# =============================================================================
class Config:
    BASE_PATH = '/kaggle/input/recodai-luc-scientific-image-forgery-detection'
    TRAIN_IMAGES_FORGED = os.path.join(BASE_PATH, 'train_images/forged')
    TRAIN_IMAGES_AUTH = os.path.join(BASE_PATH, 'train_images/authentic')
    TRAIN_MASKS = os.path.join(BASE_PATH, 'train_masks')
    TEST_IMAGES = os.path.join(BASE_PATH, 'test_images')
    SAMPLE_SUB = os.path.join(BASE_PATH, 'sample_submission.csv')

    # SIFT parameters
    SIFT_FEATURES = 5500
    SIFT_CONTRAST = 0.019
    MATCH_RATIO = 0.79
    MIN_MATCHES = 4
    RANSAC_THRESH = 5.5
    MIN_DISPLACEMENT = 23
    MAX_IMAGE_SIZE = 1600
    USE_CLAHE = True

    # Variant tweaks
    SIFT_CONFIDENCE_THRESHOLD = 0.31
    SIFT_MIN_MASK_PIXELS = 85
    SIFT_MIN_COVERAGE = 0.00035
    SIFT_MAX_COVERAGE = 0.42

    # CNN parameters
    CNN_ENABLED = True
    CNN_IMAGE_SIZE = 256
    CNN_THRESHOLD = 0.5

    # Ensemble
    ENSEMBLE_MODE = 'weighted'
    SIFT_WEIGHT = 0.75
    CNN_WEIGHT = 0.25

config = Config()
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

print(f"🎯 HYBRID ENSEMBLE - SIFT + Fast CNN")
print(f" Device: {device}")

# =============================================================================
# Dummy CNN Model (replace with your trained model)
# =============================================================================
class SimpleCNN(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = nn.Conv2d(3, 8, 3, padding=1)
        self.pool = nn.AdaptiveAvgPool2d(1)
        self.fc = nn.Linear(8,1)
    def forward(self,x):
        x = F.relu(self.conv1(x))
        x = self.pool(x)
        x = x.view(x.size(0), -1)
        x = torch.sigmoid(self.fc(x))
        return x

cnn_model = SimpleCNN().to(device)
cnn_model.eval()

# =============================================================================
# SIFT scoring
# =============================================================================
def sift_score(img_path, ref_path):
    img = cv2.imread(img_path, 0)
    ref = cv2.imread(ref_path, 0)
    if img is None or ref is None:
        return 0.0
    if config.USE_CLAHE:
        clahe = cv2.createCLAHE(clipLimit=2.0, tileGridSize=(8,8))
        img = clahe.apply(img)
        ref = clahe.apply(ref)
    sift = cv2.SIFT_create(nfeatures=config.SIFT_FEATURES, contrastThreshold=config.SIFT_CONTRAST)
    kp1, des1 = sift.detectAndCompute(img, None)
    kp2, des2 = sift.detectAndCompute(ref, None)
    if des1 is None or des2 is None:
        return 0.0
    bf = cv2.BFMatcher()
    matches = bf.knnMatch(des1, des2, k=2)
    good = []
    for m,n in matches:
        if m.distance < config.MATCH_RATIO * n.distance:
            good.append(m)
    score = len(good) / max(len(matches),1)
    return float(score)

# =============================================================================
# CNN scoring
# =============================================================================
def cnn_score(img_path):
    img = Image.open(img_path).convert('RGB').resize((config.CNN_IMAGE_SIZE, config.CNN_IMAGE_SIZE))
    img = np.array(img)/255.0
    img = torch.tensor(img).permute(2,0,1).unsqueeze(0).float().to(device)
    with torch.no_grad():
        out = cnn_model(img).item()
    return float(out)

# =============================================================================
# Generate Submission
# =============================================================================
def generate_submission():
    sub = pd.read_csv(config.SAMPLE_SUB)
    preds = []
    # Use first authentic image as reference
    ref_img_name = os.listdir(config.TRAIN_IMAGES_AUTH)[0]
    ref_path = os.path.join(config.TRAIN_IMAGES_AUTH, ref_img_name)

    for img_name in tqdm(sub['case_id']):
        img_name_str = str(img_name) + ".png"  # Convert to string and add extension
        test_path = os.path.join(config.TEST_IMAGES, img_name_str)
        s_score = sift_score(test_path, ref_path)
        c_score = cnn_score(test_path) if config.CNN_ENABLED else 0.0

        # Ensemble
        if config.ENSEMBLE_MODE == 'weighted':
            final_score = config.SIFT_WEIGHT*s_score + config.CNN_WEIGHT*c_score
        else: # voting
            final_score = 1.0 if (s_score>0.5 or c_score>config.CNN_THRESHOLD) else 0.0
        preds.append(final_score)

    sub['annotation'] = preds
    output_file = "submission.csv"
    sub.to_csv(output_file, index=False)
    print(f"✅ Submission saved to {output_file}")

# =============================================================================
# MAIN
# =============================================================================
if __name__ == "__main__":
    generate_submission()
